selenium, pyautogui, pandas를 이용해 EP에 로그인한 후 메일 앱으로 접속하고,

내 메일 목록을 긁어와 DataFrame으로 만든 후 엑셀 파일로 저장해 보세요.

In [67]:
import pandas as pd
import pyautogui as pg
from selenium import webdriver

#아이디와 비밀번호를 미리 입력받아 놓기
login_id = pg.prompt(text="EP 아이디를 입력하세요", title="EP ID 입력")
login_pw = pg.password(text="EP 비밀번호를 입력하세요", title="EP PW 입력")

#웹드라이버를 구동해 크롬 브라우저를 실행하고 EP로 접속 후 화면을 최대화함
driver = webdriver.Chrome("chromedriver")
driver.get("http://swp.posco.net")
driver.maximize_window()

In [68]:
#아이디, 비밀번호 입력 후 로그인 버튼 클릭
driver.find_element("css selector", "#username").send_keys(login_id)
driver.find_element("css selector", "#password").send_keys(login_pw)
driver.find_element("css selector", "#loginsubmit").click()

In [69]:
#이전에 접속된 정보가 있다면 브라우저에서 confirm창이 뜨므로 체크 후 확인버튼 클릭
pg.sleep(3) #경고창이 뜨기 전까지 3초 정도 대기
try:
    driver.switch_to.alert.accept()
except:
    print("경고창 없음")

In [70]:
#SSO Notice 창이 뜬 후 계속 남아 있을 경우 닫아줌
for win in driver.window_handles:
    driver.switch_to.window(win)
    if driver.title == "SSO Notice":
        driver.close()

#SSO Notice 창을 닫았다면 원래 창으로 복귀
driver.switch_to.window(driver.window_handles[0])
print(driver.title) #타이틀을 확인하여 EP창에 포커스가 맞춰져 있는지 확인

EP(Enterprise Portal)


In [71]:
#메일 아이콘의 위치를 pyautogui를 이용해 찾은 후 클릭하여 실행
#주의) pyautogui는 키보드/마우스 입력을 대신하는 것이므로 반드시 크롬창이 화면 최상단에 있어야 정상 작동함
pg.sleep(2)
email_pos = pg.locateOnScreen("data/email_icon.png", confidence=0.5)
pg.moveTo(email_pos)
pg.click()

In [72]:
#driver 객체의 포커스를 새롭게 열린 이메일 창으로 전환
driver.switch_to.window(driver.window_handles[-1])
print(driver.title) #타이틀을 확인하여 이메일 창에 포커스가 맞춰져 있는지 확인

Mail


In [73]:
mail_table = driver.find_element("css selector", ".board-list1") #메일 목록을 담고 있는 table 태그(표) 요소에 접근
mail_list = mail_table.find_elements("css selector", "tr") #메일 내용을 담고 있는 tr 태그(행) 요소에 접근

result = [] #메일 목록을 담을 빈 리스트 생성
for idx, mail in enumerate(mail_list):
    if idx > 0: #첫 번째 행은 표의 헤더 정보를 담고 있으므로 넘어감
        mail_data = mail.find_elements("css selector", "td") #행별 셀 내용을 담고 있는 td 태그(셀) 요소에 접근
        this_result = {
            "발신자" : mail_data[5].text, #5번째 td는 메일 발신자 정보
            "메일제목" : mail_data[6].text, #6번째 td는 메일 제목 정보
            "발송일시" : mail_data[7].text #7번째 td는 메일 발송일시 정보
        }
        result.append(this_result) #행별 this_result(딕셔너리) 자료를 result(리스트) 변수에 삽입

In [74]:
df = pd.DataFrame(result) #pd.DataFrame()에 result 변수를 넣으면 바로 df로 변환해 줌
df.to_excel("메일 목록 수집 결과.xlsx", index=False) #df를 엑셀로 저장

pg.alert(text="완료되었습니다.", title="알림") #작업이 모두 완료되었음을 알려줌

'OK'